<a href="https://colab.research.google.com/github/Kyveli-tsioli/hello-world/blob/main/10_01_%E2%80%93_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installation cell
%%shell
if ! command -v julia 2>&1 > /dev/null
then
    wget 'https://julialang-s3.julialang.org/bin/linux/x64/1.0/julia-1.0.5-linux-x86_64.tar.gz' \
        -O /tmp/julia.tar.gz
    tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
    rm /tmp/julia.tar.gz
fi
julia -e 'using Pkg; pkg"add IJulia; precompile;"'





////
# x has some structure of it (in terms of the features)
# underlying structure
# have x: how to represent this input for our ml model?
# as vectors?
# a good representation: need to be distinct 
# we want similar items to have similar representation
# items embedded into some space
# feature engineering: combination features/adding additional features for external resources: you help the ml system 
# the most common way make features to have some sort of indicator function:
# price of the house? high or low-> binary function -> binary representation on the features
# insurance prediction: is from uk? yes/no
# look at the space: all these are distinct 
# input as binary vector 
# sparse binary representations 
# one-hot encoding
# multi class classification: one-hot encoding 
# sparse binary features fed direcly into the weights(uncommon)
# word2vec: nlp
# 2 versions: CBOW/ Skip-gram
# 2 inputs: it projects them (calculagtes the element wise sum between each of these vectors)
# takes a weight matrix
# output: representation of whatever word is missing in the middle
# skip-gram: the inverse
# take a word, project it, try to reconstruct the context
# first instance of unsupervised learning
# no labels 
# statistical semantics 



--2019-12-03 06:33:59--  https://julialang-s3.julialang.org/bin/linux/x64/1.0/julia-1.0.5-linux-x86_64.tar.gz
Resolving julialang-s3.julialang.org (julialang-s3.julialang.org)... 151.101.2.49, 151.101.66.49, 151.101.130.49, ...
Connecting to julialang-s3.julialang.org (julialang-s3.julialang.org)|151.101.2.49|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 88706549 (85M) [application/octet-stream]
Saving to: ‘/tmp/julia.tar.gz’

/tmp/julia.tar.gz   100%[===================>]  84.60M   187MB/s    in 0.5s    

2019-12-03 06:33:59 (187 MB/s) - ‘/tmp/julia.tar.gz’ saved [88706549/88706549]

   Cloning default registries into /root/.julia/registries
   Cloning registry General from "https://github.com/JuliaRegistries/General.git"
 Resolving package versions...
 Installed VersionParsing ── v1.1.3
 Installed Conda ─────────── v1.3.0
 Installed Parsers ───────── v0.3.10
 Installed ZMQ ───────────── v1.0.0
 Installed IJulia ────────── v1.20.2
 Installed MbedTLS ───────

In [ ]:
using Pkg

pkg"add Embeddings"
pkg"add Distances"

pkg"precompile"

using Embeddings
using Distances

const w2v = load_embeddings(Word2Vec)

nothing

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]
Precompiling project...


tcmalloc: large alloc 3600007168 bytes == 0xa806000 @  0x7f6be8e1eb6b 0x7f6be8e3e379 0x7f6be87315ec 0x7f6be86fb42b 0x7f6bcb2d6ec7 0x7f6bcb2d5e5c 0x7f6be86da05c 0x7f6be86deca7 0x7f6bcb2d59b8 0x7f6bcb2d5710 0x7f6bcb2d57f8 0x7f6be86deca7 0x7f6bcb2d0af9 0x7f6bcb2d0bbb 0x7f6be86deca7 0x7f6bcb2d089a 0x7f6be86de0d6 0x7f6be8847f10 0x7f6be8847c39 0x7f6be88485bc 0x7f6be8848d3f 0x7f6be86f496c 0x7f6be884980d 0x7f6be8713ffc 0x7f6be86ef8e0 0x7f6bd6609a54 0x7f6bd660739d 0x7f6be86de0d6 0x7f6be86ec846 0x7f6be86ecea2 0x7f6bd65fd3e9


# Embeddings #

Up until this point we have dealt with continuous data (images, measurements, etc.), but a healthy amount of data one can be interested in is discrete. Think of graph structures, language, etc. These pose a challenge for neural models, why?

The vocabulary of a model for language is frequently in the millions and a big knowledge graph like Wikidata contains [66,689,671](https://www.wikidata.org/wiki/Wikidata:Statistics) entities.

What does a good representation look like in an abstract sense?:

1. Representations are **distinct**

2. Similar items have **similar** representations

If both of these statements hold, it is likely that a representation can be useful for a downstream machine learning system.

## An indicator function ##

For more “traditional” machine learning systems, we are likely to use an indicator function such as:

$$f_{id}(w) \mapsto \mathbb{N^{*}}$$

We can then create a sparse binary representation using this function:

$$g(w, i) = {\left\lbrace
                            \begin{array}{ll}
                                1 & \textrm{if }~i = f_{id}(w) \\
                                0 & \textrm{otherwise} \\
                            \end{array}\right.}$$

$$f_{sb}(w) = [g(w, 1), \ldots]$$


As a concrete “toy” example, we can imagine:

$$\mathbb{V} = \{\textrm{cat}, \textrm{dog}, \textrm{human}\}$$

$$ f_{id}(\textrm{cat}) = 1, \ldots, f_{id}(\textrm{human}) = 3$$

$$f_{sb}(\textrm{cat}) = [1, 0, 0]$$

$$f_{sb}(\textrm{dog}) = [0, 1, 0]$$

$$f_{sb}(\textrm{human}) = [0, 0, 1]$$

## A case study: word2vec ##

![](https://cdn-images-1.medium.com/max/1600/1*GkL9XH1XXSUU0RfAHKJGCg.png)

[Image source](https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf)

“If A and B have almost identical environments we say that they are synonyms” – Zellig Harris (1954)

“You shall know a word by the company it keeps” – John Rupert Firth (1957)

For some intuition, consider: “I had some _____ for breakfast today”, you instinctively know that “airplanes” is likely to be wrong, but “toast” is likely to be right.

### Formalised ###

$$ W^w \in \mathbb{R}^{|\mathbb{V}| \times d} $$

$$ w \in \mathbb{V} $$

$$ W^c \in \mathbb{R}^{|\mathbb{V}| \times d} $$

$$ c \in \mathbb{V} $$

$$ σ(x)  = \frac{1}{1 + e^{-x}} $$

$$ \mathcal{D} = \{(w_1, c_1), \ldots, (w_n, c_n)\} $$

$$ \mathcal{D}^{\prime} = \{(w_1, c_1), \ldots, (w_m, c_m)\} $$

$$ p((c, w) \in \mathcal{D}|c, w, W^{w}, W^{c}) = \sigma(W^{c}_{f_{id}(c), :} \cdot W^{w}_{f_{id}(w), :}) $$

$$ arg\max_{w,b}
    \sum_{(w, c) \in D} \textrm{log}(W^{c}_{f_{id}(c), :} \cdot W^{w}_{f_{id}(w), :})
    + \sum_{(w, c) \in D^{\prime}} \textrm{log}(-W^{c}_{f_{id}(c), :} \cdot W^{c}_{f_{id}(w), :})
$$

Why is $\mathcal{D}^{\prime}$ necessary? What happens if we remove it?

In [ ]:
const token2index = Dict(token => i for (i, token) in enumerate(w2v.vocab))
embedding(token)  = w2v.embeddings[:, token2index[token]]

nothing


# creating representations from data
# non examinable
# huge weight vector W
# embedding 
# through any text collection we have D
# collect these tuples: w together with word 1
# degine contextual window (distance of five) D'
# no labels here
# prob that this word has been observed from data 
# D' (d prime): a collection of words and context words instead of collecting the words already seen
# use sample uniformly (use frequency of words, distribution)
# w(t-2) ... similar to w(t): want
# w(t) not observed we want to be disimilar to the context of w(t-2), w(t-1) w(t+1) w(t+2)
# Wc signifies a column inseide the weight matrix
# apply the indicator function and you peak out a row outside of the weight matrix 
# Wc d dimensional vectors which are rows inside the matrices
# without negative examples our model could simply predict each word to appear with high probability
# so we need to have negative examples
# we can now learn representations


### Cosine distance ###

$$ cos(u, v) = 1 - \frac{u \cdot v}{||u|| ||v||} $$

$$ cos(u, v) \mapsto [0, 2] $$

$$ cos(u, v) = 0 \implies \textrm{identical} $$

$$ cos(u, v) = 2 \implies \textrm{opposites} $$

$$ cos(u, v) = 1 \implies \textrm{orthogonal} $$

Why are we not using something more “standard” like the Euclidian distance?

In [ ]:
distance(tokena, tokenb) = cosine_dist(embedding(tokena), embedding(tokenb))

@show distance("dog", "cat")
@show distance("cat", "dog") # Symmetric, of course.
@show distance("cat", "human")
@show distance("dog", "human")
@show distance("ape", "human");


# it gives a value between 0 and 2
# if you have value of 0 means that you have two identical vectors (same direction)
# if a value of 2 (similar but different directions)
# d directionality
# vectors of large magnitude 
# its a worst to mask differences, only changes directionality of the vectors
# thats why euclidean
# 0.32 
# pca or tsni
# takes the 1-d vector (1000 dimensional)
# and project them into 2 dimensions
# similar words have similar representations
# n=32 finds the first 32 nearest neighbors of a word (like synonyms)
# takes into account some collocations ("cats and dogs")


distance("dog", "cat") = 0.2390545f0
distance("cat", "dog") = 0.2390545f0
distance("cat", "human") = 0.75563335f0
distance("dog", "human") = 0.8109204f0
distance("ape", "human") = 0.6500703f0


![](https://cdn-images-1.medium.com/max/1600/1*vvtIsW1AblmgLkq1peKfOg.png)

[Image source](https://towardsdatascience.com/mapping-word-embeddings-with-word2vec-99a799dc9695)

## Neighbourhoods ##

In [ ]:
function neighbourhood(emb; n=32)
    nneighbours = Int[]
    maxdist     = typemax(Float32)
    for i in eachindex(w2v.vocab)
        d = cosine_dist(emb, w2v.embeddings[:, i])
        if length(nneighbours) > n && d >= maxdist
            continue
        end
        push!(nneighbours, i)
        sort!(nneighbours,
            by=j -> cosine_dist(emb, w2v.embeddings[:, j]))
        maxdist = cosine_dist(emb, w2v.embeddings[:, nneighbours[end]])
        length(nneighbours) <= n || pop!(nneighbours)
    end
    [(w2v.vocab[i], cosine_dist(emb, w2v.embeddings[:, i]))
        for i in nneighbours]
end
neighbourhood(token::String; n=32) = neighbourhood(embedding(token), n=n)

nothing

In [ ]:
neighbourhood("dog")

32-element Array{Tuple{String,Float32},1}:
 ("dog", 0.0)               
 ("dogs", 0.13195097)       
 ("puppy", 0.18935716)      
 ("pooch", 0.23726177)      
 ("cat", 0.2390545)         
 ("Rottweiler", 0.25623828) 
 ("beagle", 0.25813752)     
 ("pup", 0.25930876)        
 ("Pomeranian", 0.27613777) 
 ("canines", 0.27787548)    
 ("chihuahua", 0.2826082)   
 ("pet", 0.28352147)        
 ("schnauzer", 0.28719038)  
 ⋮                          
 ("bullmastiff", 0.31103432)
 ("sheltie", 0.322847)      
 ("Staffie", 0.32408535)    
 ("Shitzu", 0.32796305)     
 ("collie", 0.32855868)     
 ("Rottweiller", 0.33353508)
 ("kitten", 0.3340115)      
 ("cats", 0.3346824)        
 ("puppies", 0.33629328)    
 ("kennel", 0.33641297)     
 ("pug", 0.33881402)        
 ("Rotweiller", 0.33903676) 

In [ ]:
neighbourhood("england")

32-element Array{Tuple{String,Float32},1}:
 ("england", 0.0)         
 ("liverpool", 0.28275466)
 ("chelsea", 0.29395294)  
 ("fulham", 0.29792225)   
 ("tottenham", 0.3092957) 
 ("rooney", 0.3136869)    
 ("torres", 0.32362872)   
 ("ronaldo", 0.3242663)   
 ("spain", 0.32593936)    
 ("gerrard", 0.32909578)  
 ("anelka", 0.3319348)    
 ("fergie", 0.3326596)    
 ("europe", 0.335052)     
 ⋮                        
 ("newcastle", 0.3432166) 
 ("leeds", 0.34379578)    
 ("barca", 0.34468734)    
 ("madrid", 0.3449734)    
 ("everton", 0.34627277)  
 ("juve", 0.3468153)      
 ("wenger", 0.34718215)   
 ("benitez", 0.3498447)   
 ("beckham", 0.3502348)   
 ("henry", 0.35096765)    
 ("italy", 0.3517356)     
 ("pompey", 0.35407293)   

In [ ]:
neighbourhood("neuron")

32-element Array{Tuple{String,Float32},1}:
 ("neuron", 0.0)                  
 ("neurons", 0.14138311)          
 ("neuronal", 0.23898268)         
 ("neurones", 0.25595534)         
 ("synapse", 0.26050967)          
 ("synapses", 0.2702875)          
 ("neural", 0.28256142)           
 ("neurone", 0.29190993)          
 ("synaptic", 0.3098709)          
 ("cortex", 0.32931906)           
 ("Neurons", 0.34913397)          
 ("axons", 0.35264045)            
 ("axon", 0.36290532)             
 ⋮                                
 ("brain", 0.3841828)             
 ("forebrain", 0.38504547)        
 ("BDNF", 0.38742512)             
 ("astrocyte", 0.38816023)        
 ("hippocampal", 0.39035177)      
 ("hippocampus", 0.39344954)      
 ("presynaptic", 0.39414895)      
 ("astrocytes", 0.39785504)       
 ("neurogenesis", 0.39950663)     
 ("molecule", 0.39971977)         
 ("neurotransmitters", 0.39981014)
 ("MEF2", 0.40184808)             

### Word representation algebra ###

$$ f_{n}(\textrm{king}) - f_{n}(\textrm{man}) + f_{n}(\textrm{woman}) \approx f_{n}(\textrm{queen}) $$

$$ f_{n}(\textrm{Paris}) - f_{n}(\textrm{France}) + f_{n}(\textrm{Italy}) \approx f_{n}(\textrm{Rome})$$

This is notoriously difficult to replicate though…

In [ ]:
isto(ais, tob, likecto) = neighbourhood(embedding(ais) .- embedding(tob) .+ embedding(likecto))

isto (generic function with 1 method)

In [ ]:
isto("king", "man", "woman")

32-element Array{Tuple{String,Float32},1}:
 ("king", 0.2007401)         
 ("queen", 0.28818095)       
 ("monarch", 0.3810326)      
 ("princess", 0.4097572)     
 ("prince", 0.462268)        
 ("kings", 0.4763158)        
 ("queens", 0.4818864)       
 ("sultan", 0.4901408)       
 ("monarchy", 0.49125886)    
 ("throne", 0.49941933)      
 ("royal", 0.5061796)        
 ("ruler", 0.50907254)       
 ("empress", 0.5112186)      
 ⋮                           
 ("Mswati", 0.5552143)       
 ("maharaja", 0.55632824)    
 ("princesses", 0.5576925)   
 ("Princess", 0.5624594)     
 ("duchess", 0.5627598)      
 ("Queen", 0.5653622)        
 ("monarchs", 0.5656772)     
 ("handmaid", 0.5685401)     
 ("Gosakuramachi", 0.5699681)
 ("kumari", 0.57787603)      
 ("maharani", 0.57856727)    
 ("commoner", 0.57872516)    

In [ ]:
isto("Paris", "France", "Italy")
# matrix decomposition
# d hyperparameter dimensionality
# transfer learning
# weight matrices: each row is a d dimensional vector that corresponds to a word
# corresponds to a vocabulary
# it does not represent a single word, but a whole vocabulary
# rows: used when we calculate the probability of this word in the context (in the data)
# v is the number of the words?
# weights representing the words
# 

32-element Array{Tuple{String,Float32},1}:
 ("Milan", 0.27778602)  
 ("Rome", 0.2971689)    
 ("Italy", 0.3267352)   
 ("Paris", 0.3448093)   
 ("Italian", 0.4088726) 
 ("Tuscany", 0.43671834)
 ("Bologna", 0.43916416)
 ("Sicily", 0.44036168) 
 ("Genoa", 0.46910983)  
 ("Milanese", 0.4694308)
 ("Rimini", 0.48378998) 
 ("Venice", 0.4858547)  
 ("ANSA", 0.4889533)    
 ⋮                      
 ("Pisa", 0.499672)     
 ("Liguria", 0.5008726) 
 ("Sassari", 0.50194716)
 ("Brianza", 0.50275654)
 ("Sempione", 0.5056912)
 ("Pietro", 0.5061012)  
 ("Pomezia", 0.50707614)
 ("Giorgio", 0.5097326) 
 ("Vicenza", 0.5102454) 
 ("Gianni", 0.51183736) 
 ("Pistoia", 0.51197916)
 ("Jacopo", 0.51241815) 

![](https://samyzaf.com/ML/nlp/word2vec2.png)

[Image source](https://samyzaf.com/ML/nlp/nlp.html)

Why and how would these relationships be preserved in the embeddings? Think, what do they capture?

**Beware** when playing around with these models though, they are a reflection of the underlying data and very well mirrors the worst of us as a species in our racism, sexism, etc.

## Some closing comments and questions… ##

The same line of thinking case be used for graph embeddings, etc. I also find it helpful to think about what an MLP or CNN achieves in a similar way.

1. How do we handle sequences?

2. Our representations are context independent, is this a problem?

3. Can we imagine loss functions akin to word2vec for other areas?

4. From your Linear Algebra, does $W^w$ and $W^c$ remind you of something? Does this hint at another way to learn embeddings?

## Extracurricular reading ##

1. [“Distributed Representations of Words and Phrases and their Compositionality”](https://papers.nips.cc/paper/5021-distributed-representations-of-words-andphrases) by Mikolov et al. (2013)

2. [“word2vec Explained: deriving Mikolov et al.’s negative-sampling word-embedding method”](https://arxiv.org/abs/1402.3722) by Goldberg and Levy (2014). I *highly* recommend to **read this in parallel with Mikolov et al. (2013)**.